# 小米9.02笔试题
## 参考面经：





1. 小米从N个花架上挑选4种花，每个花架上均有这4种花但是价格不一，小米想花完1000块，求有多少种挑选方案？


最基础的做法，回溯（DFS），会超时

In [8]:
N = 3
case1 = [
    [250, 250, 250, 250],
    [156, 201, 205, 400],
    [205, 190, 100, 250]
]

def solve(arr, n, m):
    ans = 0

    def dfs(j, rest):
        nonlocal ans
        if rest == 0:
            ans += 1
        if j >= 4:
            return
        for i in range(n):
            dfs(j+1, rest-arr[i][j])

    dfs(0, 1000)
    return ans

print(solve(case1, N, 1000))

4


加入记忆化搜索后，如果前j种花达到某一金额cnt，剩下的就是寻找j+1~4的花里金额为1000-cnt的组合数。不知道能不能ac，这个内存占用会比较高

In [16]:
N = 3
case1 = [
    [250, 250, 250, 250],
    [156, 201, 205, 400],
    [205, 190, 100, 250]
]

def solve(arr, n, m):
    mem = {}
    
    def dfs(j, rest):
        total = 0
        if rest == 0:
            total += 1
        if j >= 4:
            return total
        for i in range(n):
            if rest-arr[i][j] < 0:
                continue
            if (j+1, rest-arr[i][j]) not in mem:
                mem[(j+1, rest-arr[i][j])] = dfs(j+1, rest-arr[i][j])
            total += mem[(j+1, rest-arr[i][j])]
        return total

    ans = dfs(0, 1000)
    return ans

print(solve(case1, N, 1000))

4


2. 给定真实文本和输出文本，在编辑距离计算的基础上，要求计算出插入、删除和替换的次数。
 
    实际场景中，编辑距离对应的插入次数、删除次数、替换次数有可能不唯一，本题目不考虑不唯一的情况，即假定所有输入样例只有唯一解

    输入描述：

    输入为两行字符串。字符串可能包含英文大小写字母、数字、英文标点符号、空格等字符。字符串长度小于等于1000.第一行为真实文本的字符序列，第二行为语音识别输出文本的字符序列

    输出猫述：

    三行整数，分别表示插入错误、删除错误、替换错误的次数

首先，回顾一下编辑距离最初始的做法：

In [21]:
origin_sentence = "Xmi cpany."
result_sentence = "XMi c."

def minEditDistanceOperation(s1, s2):
    n, m = len(s1), len(s2)
    # dp[i][j]为s1[:i]与s2[:j]的最小编辑距离
    dp = [[0 for i in range(m+1)] for j in range(n+1)]
    for i in range(n):
        dp[i][0] = i
    for j in range(m):
        dp[0][j] = j

    for i in range(1, n+1):
        for j in range(1, m+1):
            if s1[i-1] == s2[j-1]:
                dp[i][j] = dp[i-1][j-1]
            else:
                dp[i][j] = min(dp[i-1][j], dp[i][j-1], dp[i-1][j-1]) + 1 
    for i in dp:print(i)
    return dp[n][m]

print(minEditDistanceOperation(origin_sentence, result_sentence))

[0, 1, 2, 3, 4, 5, 0]
[1, 0, 1, 2, 3, 4, 1]
[2, 1, 1, 2, 3, 4, 2]
[3, 2, 2, 1, 2, 3, 3]
[4, 3, 3, 2, 1, 2, 3]
[5, 4, 4, 3, 2, 1, 2]
[6, 5, 5, 4, 3, 2, 2]
[7, 6, 6, 5, 4, 3, 3]
[8, 7, 7, 6, 5, 4, 4]
[9, 8, 8, 7, 6, 5, 5]
[0, 1, 2, 3, 4, 5, 5]
5


观察dp矩阵，可以通过后向回溯或前向记录辅助数组来统计：

In [6]:
origin_sentence = "Xmi cpany."
result_sentence = "XMi c."

def minEditDistanceOperation(s1, s2):
    n, m = len(s1), len(s2)
    # dp[i][j]为s1[:i]与s2[:j]的最小编辑距离
    dp = [[0 for i in range(m+1)] for j in range(n+1)]
    op = [['_' for i in range(m+1)] for j in range(n+1)]
    for i in range(1, n+1):
        dp[i][0] = i
        op[i][0] = s1[i-1]
    for j in range(1, m+1):
        dp[0][j] = j
        op[0][j] = s2[j-1]
    
    for i in range(1, n+1):
        for j in range(1, m+1):
            if s1[i-1] == s2[j-1]:
                dp[i][j] = dp[i-1][j-1]
                op[i][j] = str(dp[i][j]) + ' ↘'
            else:
                dp[i][j] = min(dp[i-1][j], dp[i][j-1], dp[i-1][j-1]) + 1 
                if dp[i][j] == dp[i-1][j]+1:
                    op[i][j] = str(dp[i][j]) + ' ↓'
                elif dp[i][j] == dp[i][j-1]+1:
                    op[i][j] = str(dp[i][j]) + ' →'
                else:
                    op[i][j] = str(dp[i][j]) + ' ↘'
    
    for i in dp:print(i)

    insert, delete, replace = 0, 0, 0
    i, j = n, m
    while i > 0 or j > 0:
        if op[i][j][-1] == '↘':
            op[i][j] += '*'
            if dp[i][j] == dp[i-1][j-1] + 1:
                replace += 1
            i -= 1
            j -= 1
        elif op[i][j][-1] == '↓':
            op[i][j] += '*'
            delete += 1
            i -= 1
        elif op[i][j][-1] == '→':
            op[i][j] += '*'
            insert += 1
            j -= 1
    cnt = 0
    for i in op:
        if cnt == 0:
            print(i[0], end='\t ')
            print('    \t'.join(i[1:]))
        else:
            print('\t'.join(i))
        cnt += 1
        # print('\n')
    print("insert:{}\ndelete:{}\nreplace:{}\ntotal:{}".format(insert, delete, replace,dp[n][m]))
    return dp[n][m]

print(minEditDistanceOperation(origin_sentence, result_sentence))

[0, 1, 2, 3, 4, 5, 6]
[1, 0, 1, 2, 3, 4, 5]
[2, 1, 1, 2, 3, 4, 5]
[3, 2, 2, 1, 2, 3, 4]
[4, 3, 3, 2, 1, 2, 3]
[5, 4, 4, 3, 2, 1, 2]
[6, 5, 5, 4, 3, 2, 2]
[7, 6, 6, 5, 4, 3, 3]
[8, 7, 7, 6, 5, 4, 4]
[9, 8, 8, 7, 6, 5, 5]
[10, 9, 9, 8, 7, 6, 5]
_	 X    	M    	i    	     	c    	.
X	0 ↘*	1 →	2 →	3 →	4 →	5 →
m	1 ↓	1 ↘*	2 →	3 →	4 →	5 →
i	2 ↓	2 ↓	1 ↘*	2 →	3 →	4 →
 	3 ↓	3 ↓	2 ↓	1 ↘*	2 →	3 →
c	4 ↓	4 ↓	3 ↓	2 ↓	1 ↘*	2 →
p	5 ↓	5 ↓	4 ↓	3 ↓	2 ↓*	2 ↘
a	6 ↓	6 ↓	5 ↓	4 ↓	3 ↓*	3 ↓
n	7 ↓	7 ↓	6 ↓	5 ↓	4 ↓*	4 ↓
y	8 ↓	8 ↓	7 ↓	6 ↓	5 ↓*	5 ↓
.	9 ↓	9 ↓	8 ↓	7 ↓	6 ↓	5 ↘*
insert:0
delete:4
replace:1
total:5
5


通过方向标记，我们能很方便的进行回溯最短的路径并可视化，同时统计不同方向所对应的编辑操作个数。
不过上述方向矩阵主要是帮助理解，真笔试的时候最好直接回溯就行，如下：

In [7]:
origin_sentence = "Xmi cpany."
result_sentence = "XMi c."

def minEditDistanceOperation(s1, s2):
    n, m = len(s1), len(s2)
    # dp[i][j]为s1[:i]与s2[:j]的最小编辑距离
    dp = [[0 for i in range(m+1)] for j in range(n+1)]
    for i in range(1, n+1):
        dp[i][0] = i
    for j in range(1, m+1):
        dp[0][j] = j
    
    for i in range(1, n+1):
        for j in range(1, m+1):
            if s1[i-1] == s2[j-1]:
                dp[i][j] = dp[i-1][j-1]
            else:
                dp[i][j] = min(dp[i-1][j], dp[i][j-1], dp[i-1][j-1]) + 1 
    
    for i in dp:print(i)

    insert, delete, replace = 0, 0, 0
    i, j = n, m
    while i > 0 or j > 0:
        if dp[i][j] == dp[i-1][j-1] + 1:
            replace += 1
            i -= 1
            j -= 1
        elif dp[i][j] == dp[i-1][j] + 1:
            delete += 1
            i -= 1
        elif dp[i][j] == dp[i][j-1] + 1:
            insert += 1
            j -= 1
        elif dp[i][j] == dp[i-1][j-1]:
            i -= 1
            j -= 1

    print("insert:{}\ndelete:{}\nreplace:{}\ntotal:{}".format(insert, delete, replace,dp[n][m]))
    return dp[n][m]

print(minEditDistanceOperation(origin_sentence, result_sentence))

[0, 1, 2, 3, 4, 5, 6]
[1, 0, 1, 2, 3, 4, 5]
[2, 1, 1, 2, 3, 4, 5]
[3, 2, 2, 1, 2, 3, 4]
[4, 3, 3, 2, 1, 2, 3]
[5, 4, 4, 3, 2, 1, 2]
[6, 5, 5, 4, 3, 2, 2]
[7, 6, 6, 5, 4, 3, 3]
[8, 7, 7, 6, 5, 4, 4]
[9, 8, 8, 7, 6, 5, 5]
[10, 9, 9, 8, 7, 6, 5]
insert:0
delete:4
replace:1
total:5
5
